In [31]:
!pip install scikit-learn

You should consider upgrading via the 'C:\Users\David\PycharmProjects\big_data_tfm\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_all = pd.read_csv("pisos.csv")

df_expensive = df_all[df_all['price_euro'] > 350000]
df_cheap = df_all[df_all['price_euro'] <= 350000]


# categorical_cols = ['title', 'location', 'city', 'description', 'link', 'updated_date',
#        'planta', 'referencia', 'carpinteria_exterior', 'garaje', 'trastero',
#        'ascensor', 'photos', 'exterior', 'conservacion', 'balcon',
#        'armarios_empotrados', 'terraza', 'piscina', 'orientacion',
#        'gastos_de_comunidad', 'amueblado',
#        'adaptado_a_personas_con_movilidad_reducida', 'tipo_suelo', 'jardin',
#        'telefono', 'cocina_equipada', 'comedor', 'portero_automatico',
#        'antiguedad', 'carpinteria_interior', 'soleado', 'cocina', 'urbanizado',
#        'calle_alumbrada', 'calle_asfaltada', 'agua', 'tipo_de_casa',
#        'aire_acondicionado', 'lavadero', 'puerta_blindada', 'interior',
#        'se_aceptan_mascotas', 'vidrios_dobles', 'chimenea',
#        'sistema_de_seguridad', 'calefaccion', 'gas', 'luz', 'alcantarillado',
#        'no_se_aceptan_mascotas', 'exterior_summary', 'vidrios_dobles_summary',
#        'adaptado_a_personas_con_movilidad_reducida_summary',
#        'puerta_blindada_summary', 'ascensor_summary', 'balcon_summary',
#        'portero_automatico_summary', 'garaje_summary', 'comedor_summary',
#        'terraza_summary', 'jardin_summary', 'armarios_empotrados_summary',
#        'aire_acondicionado_summary', 'trastero_summary', 'piscina_summary',
#        'chimenea_summary', 'lavadero_summary', 'urbanizado_summary',
#        'calle_alumbrada_summary', 'calle_asfaltada_summary', 'soleado_summary',
#        'gas_summary', 'sistema_de_seguridad_summary', 'interior_summary',
#        'alcantarillado_summary', 'amueblado_summary',
#        'cocina_equipada_summary', 'mascotas_summary',
#        'carpinteria_exterior_cleaned', 'tipo_suelo_summary',
#        'calefaccion_summary', 'cocina_summary', 'orientacion_summary',
#        'agua_summary', 'type', 'esquina', 'esquina_summary']

# categorical_clean = ['conservacion', 'antiguedad', 'tipo_de_casa', 'alcantarillado', 
#                      'exterior_summary', 'vidrios_dobles_summary', 'adaptado_a_personas_con_movilidad_reducida_summary', 
#                      'puerta_blindada_summary', 'ascensor_summary', 'balcon_summary', 'portero_automatico_summary', 
#                      'garaje_summary', 'comedor_summary', 'terraza_summary', 
#                      'jardin_summary', 'armarios_empotrados_summary', 'aire_acondicionado_summary', 
#                      'trastero_summary', 'piscina_summary', 'chimenea_summary', 
#                      'lavadero_summary', 'urbanizado_summary', 'calle_alumbrada_summary', 
#                      'calle_asfaltada_summary', 'soleado_summary', 'gas_summary', 
#                      'sistema_de_seguridad_summary', 'interior_summary', 'alcantarillado_summary', 
#                      'amueblado_summary', 'cocina_equipada_summary', 'mascotas_summary', 
#                      'carpinteria_exterior_cleaned', 'tipo_suelo_summary', 'calefaccion_summary', 
#                      'cocina_summary', 'orientacion_summary', 'agua_summary', 'gas_summary' 
                     # 'type', 'esquina_summary']

categorical_to_fill_NO = ['exterior_summary', 'vidrios_dobles_summary', 'adaptado_a_personas_con_movilidad_reducida_summary',
                         'puerta_blindada_summary', 'ascensor_summary', 'balcon_summary', 'portero_automatico_summary',
                        'garaje_summary', 'comedor_summary', 'terraza_summary', 
                        'jardin_summary', 'armarios_empotrados_summary', 'aire_acondicionado_summary', 
                          'trastero_summary', 'piscina_summary', 'chimenea_summary', 
                           'lavadero_summary', 'soleado_summary', 'gas_summary',
                          'amueblado_summary', 'cocina_equipada_summary', 'calefaccion_summary',
                         ]
categorical_to_fill_DESCONOCIDO = ['city', 'location', 'conservacion', 'antiguedad', 'tipo_de_casa', 'urbanizado_summary', 
                                  'calle_alumbrada_summary', 'calle_asfaltada_summary', 'interior_summary', 
                                  'mascotas_summary', 'carpinteria_exterior_cleaned', 'tipo_suelo_summary',
                                  'cocina_summary', 'orientacion_summary', 'orientacion_summary', 
                                  'type']

categorical = categorical_to_fill_NO + categorical_to_fill_DESCONOCIDO

def clean_data(origin_df):
    df = origin_df.copy()

    
    numerical = ['price_euro', 'old_price_euro', 'superficie_construida_m2', 
                'superficie_util_m2', 'habitaciones', 'banos', 
                'superficie_solar_m2', 'gastos_de_comunidad_cleaned']
    # Fill NaN values with 'NO' for selected categorical columns
    for col in categorical_to_fill_NO:
        df[col].fillna('NO', inplace=True)
    
    for col in categorical_to_fill_DESCONOCIDO:
        df[col].fillna('DESCONOCIDO', inplace=True)
    
    

    df = df[categorical + numerical]

    # Create a mask for rows where both columns superficie are not null
    mask = df['superficie_util_m2'].notna() & df['superficie_construida_m2'].notna()
    # Calculate the percentage difference for these rows
    percentage_diffs = (df.loc[mask, 'superficie_construida_m2'] - df.loc[mask, 'superficie_util_m2']) / df.loc[mask, 'superficie_construida_m2']
    # Calculate the average percentage
    average_percentage = percentage_diffs.mean()
    mask = df['superficie_construida_m2'].isna() & df['superficie_util_m2'].notna()
    df.loc[mask, 'superficie_construida_m2'] = df.loc[mask, 'superficie_util_m2'] * (1 + average_percentage)
    median_value = df['superficie_construida_m2'].median()
    df['superficie_construida_m2'].fillna(median_value, inplace=True)
    
    mask = df['superficie_util_m2'].isna()
    df.loc[mask, 'superficie_util_m2'] = df.loc[mask, 'superficie_construida_m2'] * (1 - average_percentage)

    df['superficie_solar_m2'].fillna(0, inplace=True)

    df.dropna(subset=['price_euro'], inplace=True)
    df['old_price_euro'].fillna(df['price_euro'], inplace=True)

    def convert_to_cat(val):
        if pd.isna(val):
            return "DESCONOCIDO"
        elif val <= 6:
            return str(int(val))
        else:
            return "7 or more"
    
    def convert_gastos(val):
        if pd.isna(val):
            return "DESCONOCIDO"
        elif val == 0:
            return "0"
        elif 0 < val <= 20:
            return "0-20"
        elif 20 < val <= 40:
            return "20-40"
        elif 40 < val <= 60:
            return "40-60"
        elif 60 < val <= 80:
            return "60-80"
        elif 80 < val <= 100:
            return "80-100"
        elif 100 < val <= 120:
            return "100-120"
        elif 120 < val <= 140:
            return "120-140"
        elif 140 < val <= 160:
            return "140-160"
        elif 160 < val <= 180:
            return "160-180"
        elif 180 < val <= 200:
            return "180-200"
        elif val > 200:
            return "200+"
    
    df['gastos_de_comunidad_cleaned'] = df['gastos_de_comunidad_cleaned'].apply(convert_gastos)
    df['habitaciones'] = df['habitaciones'].apply(convert_to_cat)
    df['banos'] = df['banos'].apply(convert_to_cat)

    nan_counts = df[numerical+categorical].isna().sum()
    assert(nan_counts.sum() == 0)
    return df
    

C:\Users\David\AppData\Local\Temp\ipykernel_4480\1079593130.py:5: DtypeWarning: Columns (58,84,96,97) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv("pisos.csv")


In [33]:
print(len(df_expensive), len(df_cheap))
df_cheap = clean_data(df_cheap)
df_expensive = clean_data(df_expensive)

26680 140076


In [34]:
def create_model(df, city, model_type):
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
    from sklearn.svm import SVR
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.metrics import mean_squared_error
    
    

    
    # Drop the target and other unwanted columns
    X = df.drop(columns=['price_euro', 'old_price_euro'])
    
    # One-hot encode the categorical columns
    X_encoded = pd.get_dummies(X, columns=categorical+['habitaciones','banos','gastos_de_comunidad_cleaned'])
    
    # Define the target
    y = df['price_euro']
    
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

    if model_type == "RandomForest":
        model = RandomForestRegressor(n_estimators=100, random_state=42)
    elif model_type == "GradientBoosting":
        model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    elif model_type == "AdaBoost":
        model = AdaBoostRegressor(n_estimators=100, random_state=42)
    elif model_type == "ExtraTrees":
        model = ExtraTreesRegressor(n_estimators=100, random_state=42)
    elif model_type == "DecisionTree":
        model = DecisionTreeRegressor(random_state=42)
    elif model_type == "LinearRegression":
        model = LinearRegression()
    elif model_type == "Ridge":
        model = Ridge()
    elif model_type == "Lasso":
        model = Lasso()
    elif model_type == "ElasticNet":
        model = ElasticNet()
    elif model_type == "SVR":
        model = SVR()
    elif model_type == "KNN":
        model = KNeighborsRegressor()

    base_model = model
    base_model.fit(X_train, y_train)

    
    # Predict using the best model
    y_pred = base_model.predict(X_test)

    def plot_true_to_predicted(y_test, y_pred):
        true_values = y_test.tolist()
        predicted_values = y_pred.tolist()
        plt.figure(figsize=(10, 6))
        plt.scatter(true_values, predicted_values, alpha=0.5)
        plt.xlabel("True Values")
        plt.ylabel("Predicted Values")
        plt.title(f"True vs. Predicted Values for {city}")
        plt.plot([min(true_values), max(true_values)], 
                 [min(true_values), max(true_values)], 'r')
        plt.show()

    # Check if the model has the feature_importances_ attribute
    if hasattr(base_model, 'feature_importances_'):
        feature_importances = base_model.feature_importances_
        importance_df = pd.DataFrame({
            'feature': X_encoded.columns,
            'importance': feature_importances
        }).sort_values(by='importance', ascending=False)
        
        print(importance_df)
    else:
        print(f"{model_type} does not have feature_importances_ attribute.")

    # Calculate the mean squared error
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    # Statistics of price_euro
    mean_price = df['price_euro'].mean()
    std_price = df['price_euro'].std()


    return base_model, rmse, mean_price

def generate_models(unique_cities, model_type="RandomForest"):

    models = {}  # To store trained models for each city
    rmse_scores = {}  # To store RMSE scores for each city
    mean_prices = {}
    
    for city in unique_cities:
        
        city_df_cheap = df_cheap[df_cheap['city'] == city].copy()
        
        cheap_model, cheap_rmse, cheap_mean_price = create_model(city_df_cheap, city, model_type)
        # Store the trained model and RMSE
    
        print("Cheap model -----------")
        print(f"City: {city}, RMSE: {cheap_rmse:.2f}, mean_price: {cheap_mean_price}")
    
        city_df_expensive = df_expensive[df_expensive['city'] == city].copy()
        
        expensive_model, expensive_rmse, expensive_mean_price = create_model(city_df_expensive, city, model_type)
    
    
        print("Expensive model ----------")
        print(f"City: {city}, RMSE: {expensive_rmse:.2f}, mean_price: {expensive_mean_price}")
    
        models[city] = {'cheap': cheap_model, 'expensive': expensive_model}
        rmse_scores[city] = {'cheap': cheap_rmse, 'expensive': expensive_rmse}
        mean_prices[city] = {'cheap': cheap_mean_price, 'expensive': expensive_mean_price}

    rmse_scores_relative = { k: {'cheap': rmse_scores[k]['cheap']/mean_prices[k]['cheap'],
                            'expensive': rmse_scores[k]['expensive']/mean_prices[k]['expensive']
                            } for k in rmse_scores.keys()}

    return rmse_scores_relative, models






In [35]:
def get_best_models(rmse_results):
    """
    Determine the best model for each city and category based on RMSE.

    rmse_results: Nested dictionary where the first key is model type, 
                  the second key is city name, and the value is a dictionary 
                  with 'cheap' and 'expensive' RMSEs.
    """
    best_models = {}
    
    for city in rmse_results[next(iter(rmse_results))].keys():  # use next(iter()) to get the first model type
        best_models[city] = {
            "cheap": min([(model, data[city]['cheap']) for model, data in rmse_results.items()], key=lambda x: x[1]),
            "expensive": min([(model, data[city]['expensive']) for model, data in rmse_results.items()], key=lambda x: x[1])
        }
    
    return best_models

# Main code
model_types = ["RandomForest", "GradientBoosting", "AdaBoost", "ExtraTrees", "DecisionTree","LinearRegression","Ridge","Lasso","ElasticNet","SVR","KNN"]
rmse_results_all_models = {}

for model_type in model_types:
    rmse_scores_relative, models = generate_models(unique_cities=df_all['city'].unique(), model_type=model_type)
    rmse_results_all_models[model_type] = rmse_scores_relative



                                  feature  importance
0                superficie_construida_m2    0.103010
1                      superficie_util_m2    0.080453
21                     terraza_summary_NO    0.060379
22                    terraza_summary_YES    0.051135
315                               banos_1    0.038340
..                                    ...         ...
321                       banos_7 or more    0.000000
137              location_La Jaca (Arico)    0.000000
141      location_La Perdoma (La Orotava)    0.000000
179              location_Pájara (Güímar)    0.000000
167  location_Montero (Icod de Los Vinos)    0.000000

[334 rows x 2 columns]
Cheap model -----------
City: tenerife, RMSE: 51122.37, mean_price: 198173.72724113968
                                        feature  importance
1                            superficie_util_m2    0.265147
83                 location_Costa Adeje (Adeje)    0.199467
0                      superficie_construida_m2    0.082897
1

c:\users\david\pycharmprojects\big_data_tfm\venv\Lib\site-packages\sklearn\linear_model\_ridge.py:250: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


Ridge does not have feature_importances_ attribute.
Expensive model ----------
City: soria, RMSE: 40217.14, mean_price: 456779.5918367347
Ridge does not have feature_importances_ attribute.
Cheap model -----------
City: oviedo, RMSE: 39862.61, mean_price: 144378.32700135684
Ridge does not have feature_importances_ attribute.
Expensive model ----------
City: oviedo, RMSE: 252137.72, mean_price: 586190.5975609756
Ridge does not have feature_importances_ attribute.
Cheap model -----------
City: isla_de_mallorca, RMSE: 55118.52, mean_price: 241885.32208588958
Ridge does not have feature_importances_ attribute.
Expensive model ----------
City: isla_de_mallorca, RMSE: 1507361.03, mean_price: 1145980.1625514403
Ridge does not have feature_importances_ attribute.
Cheap model -----------
City: pamplona_iruna, RMSE: 37651.17, mean_price: 221485.52
Ridge does not have feature_importances_ attribute.
Expensive model ----------
City: pamplona_iruna, RMSE: 208135.19, mean_price: 520361.9029850746
Ri

c:\users\david\pycharmprojects\big_data_tfm\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.091e+11, tolerance: 1.297e+09
  model = cd_fast.enet_coordinate_descent(


Lasso does not have feature_importances_ attribute.
Cheap model -----------
City: tenerife, RMSE: 55854.96, mean_price: 198173.72724113968


...

c:\users\david\pycharmprojects\big_data_tfm\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.527e+11, tolerance: 2.758e+08
  model = cd_fast.enet_coordinate_descent(


ElasticNet does not have feature_importances_ attribute.
Expensive model ----------
City: vigo, RMSE: 223600.57, mean_price: 528930.8510638297


c:\users\david\pycharmprojects\big_data_tfm\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.498e+12, tolerance: 7.769e+08
  model = cd_fast.enet_coordinate_descent(


ElasticNet does not have feature_importances_ attribute.
Cheap model -----------
City: navarra_nafarroa, RMSE: 64840.48, mean_price: 158725.79296615793
ElasticNet does not have feature_importances_ attribute.
Expensive model ----------
City: navarra_nafarroa, RMSE: 4441333.50, mean_price: 570944.9206349206
SVR does not have feature_importances_ attribute.
Cheap model -----------
City: tenerife, RMSE: 74818.15, mean_price: 198173.72724113968
SVR does not have feature_importances_ attribute.
Expensive model ----------
City: tenerife, RMSE: 963709.71, mean_price: 924522.5831889082
SVR does not have feature_importances_ attribute.
Cheap model -----------
City: lleida, RMSE: 76689.47, mean_price: 125114.37211686315
SVR does not have feature_importances_ attribute.
Expensive model ----------
City: lleida, RMSE: 503896.79, mean_price: 726578.3005464481
SVR does not have feature_importances_ attribute.
Cheap model -----------
City: fuerteventura, RMSE: 71905.04, mean_price: 186508.1372141372
S

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 3, n_neighbors = 5

In [36]:
# Printing the results:
best_models_for_each_city = get_best_models(rmse_results_all_models)
for city, models in best_models_for_each_city.items():
    print(f"City: {city}")
    print(f"Best Cheap Model: {models['cheap'][0]} with RMSE of {models['cheap'][1]:.4f}")
    print(f"Best Expensive Model: {models['expensive'][0]} with RMSE of {models['expensive'][1]:.4f}")
    print("------------------------")

City: tenerife
Best Cheap Model: RandomForest with RMSE of 0.2580
Best Expensive Model: GradientBoosting with RMSE of 0.6022
------------------------
City: lleida
Best Cheap Model: RandomForest with RMSE of 0.3598
Best Expensive Model: RandomForest with RMSE of 0.5344
------------------------
City: fuerteventura
Best Cheap Model: RandomForest with RMSE of 0.2107
Best Expensive Model: ElasticNet with RMSE of 0.3789
------------------------
City: asturias
Best Cheap Model: Ridge with RMSE of 0.3554
Best Expensive Model: GradientBoosting with RMSE of 0.4068
------------------------
City: formentera
Best Cheap Model: RandomForest with RMSE of 0.0000
Best Expensive Model: DecisionTree with RMSE of 0.9664
------------------------
City: tarragona
Best Cheap Model: RandomForest with RMSE of 0.2758
Best Expensive Model: ExtraTrees with RMSE of 0.3737
------------------------
City: almeria
Best Cheap Model: RandomForest with RMSE of 0.3327
Best Expensive Model: GradientBoosting with RMSE of 0.25